#### 데이터 불러오기

In [3]:
import numpy as np
import os
from glob import glob
from tqdm import tqdm

'''
Dataset from https://gomocup.org/results/
'''
game_rule1 = 'freestyle' #2009년 데이터는 소문자로 시작하더라고요.. 
game_rule2 = 'Freestyle' # Freestyle, Fastgame, Standard, Renju
#base_path = '/Users/brad/Downloads/gomocup2019results' 

base_path1 = '/Users/hanjunghwa/Downloads/gomocup2019results'
base_path2 = '/Users/hanjunghwa/Downloads/gomocup2009results'

#output_path = os.path.join('dataset', os.path.basename(base_path))
#os.makedirs(output_path, exist_ok=True)

output_path1 = os.path.join('dataset', os.path.basename(base_path1))
os.makedirs(output_path1, exist_ok=True)

output_path2 = os.path.join('dataset', os.path.basename(base_path2))
os.makedirs(output_path2, exist_ok=True)

#file_list = glob(os.path.join(base_path, '%s*/*.psq' % (game_rule, )))

file_list1 = glob(os.path.join(base_path1, '%s*/*.psq' % (game_rule2, )))
file_list2 = glob(os.path.join(base_path2, '%s*/*.psq' % (game_rule1, )))
file_list = file_list1 + file_list2

In [4]:
#read_dataset.py에서 데이터 잘 불러왔는지 확인용 
print(f"2009년 freestyle 데이터 : {len(file_list1)}")
print(f"2019년 freestyle 데이터 : {len(file_list2)}")
print(f"학습할 데이터 : {len(file_list)}")
print(output_path1)
print(output_path2)

2009년 freestyle 데이터 : 10224
2019년 freestyle 데이터 : 864
학습할 데이터 : 11088
dataset/gomocup2019results
dataset/gomocup2009results


#### 데이터 전처리

In [5]:
for index, file_path in enumerate(tqdm(file_list)):
    with open(file_path, 'r') as f:
        lines = f.read().splitlines() 

    w, h = lines[0].split(' ')[1].strip(',').split('x')
    w, h = int(w), int(h)

    lines = lines[1:]

    inputs, outputs = [], []
    board = np.zeros([h, w], dtype=np.int8)

    for i, line in enumerate(lines):
        if ',' not in line:
            break

        x, y, t = np.array(line.split(','), np.int8)

        if i % 2 == 0:
            player = 1
        else:
            player = 2

        input = board.copy().astype(np.int8)
        input[(input != player) & (input != 0)] = -1
        input[(input == player) & (input != 0)] = 1

        output = np.zeros([h, w], dtype=np.int8)
        output[y-1, x-1] = 1

        # augmentation
        # rotate 4 x flip 3 = 12
        for k in range(4):
            input_rot = np.rot90(input, k=k)
            output_rot = np.rot90(output, k=k)

            inputs.append(input_rot)
            outputs.append(output_rot)

            inputs.append(np.fliplr(input_rot))
            outputs.append(np.fliplr(output_rot))

            inputs.append(np.flipud(input_rot))
            outputs.append(np.flipud(output_rot))

        # update board
        board[y-1, x-1] = player

    # save dataset
    np.savez_compressed(os.path.join(output_path1, '%s.npz' % (str(index).zfill(5))), inputs=inputs, outputs=outputs)

100%|████████████████████████████████████| 11088/11088 [00:56<00:00, 195.85it/s]


#### cnn 학습

In [5]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, models
import numpy as np
from sklearn.model_selection import train_test_split
from glob import glob
import os
from tqdm import tqdm
from datetime import datetime
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [6]:
w, h = 40, 40
base_path = os.path.join('dataset', '*/*.npz')

file_list = glob(base_path)

x_data, y_data = [], []
for file_path in tqdm(file_list):
    data = np.load(file_path)
    x_data.extend(data['inputs'])
    y_data.extend(data['outputs'])

x_data = np.array(x_data, np.float32).reshape((-1, h, w, 1))
y_data = np.array(y_data, np.float32).reshape((-1, h * w))

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=2020)

del x_data, y_data

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

100%|████████████████████████████████████| 11088/11088 [00:13<00:00, 852.01it/s]


(1219423, 40, 40, 1) (1219423, 1600)
(304856, 40, 40, 1) (304856, 1600)


In [7]:
model = models.Sequential([
    layers.Conv2D(64, 7, activation='relu', padding='same', input_shape=(h, w, 1)),
    layers.Conv2D(128, 7, activation='relu', padding='same'),
    layers.Conv2D(256, 7, activation='relu', padding='same'),
    layers.Conv2D(128, 7, activation='relu', padding='same'),
    layers.Conv2D(64, 7, activation='relu', padding='same'),
    layers.Conv2D(1, 1, activation=None, padding='same'),
    layers.Reshape((h * w,)),
    layers.Activation('sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['acc']
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 40, 40, 64)        3200      
                                                                 
 conv2d_1 (Conv2D)           (None, 40, 40, 128)       401536    
                                                                 
 conv2d_2 (Conv2D)           (None, 40, 40, 256)       1605888   
                                                                 
 conv2d_3 (Conv2D)           (None, 40, 40, 128)       1605760   
                                                                 
 conv2d_4 (Conv2D)           (None, 40, 40, 64)        401472    
                                                                 
 conv2d_5 (Conv2D)           (None, 40, 40, 1)         65        
                                                                 
 reshape (Reshape)           (None, 1600)              0

In [1]:
import torch
device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu')
print (f"PyTorch version:{torch.__version__}") # 1.12.1 이상
print(f"MPS 장치를 지원하도록 build 되었는지: {torch.backends.mps.is_built()}") # True 여야 합니다.
print(f"MPS 장치가 사용 가능한지: {torch.backends.mps.is_available()}") # True 여야 합니다.
!python -c 'import platform;print(platform.platform())'
print(f"device: {device}")

PyTorch version:2.0.0
MPS 장치를 지원하도록 build 되었는지: True
MPS 장치가 사용 가능한지: True
macOS-10.16-x86_64-i386-64bit
device: mps:0


In [ ]:
start_time = datetime.now().strftime('%Y%m%d_%H%M%S')
os.makedirs('models', exist_ok=True)

with tf.device('/gpu:0'):
    model.fit(
        x=x_train,
        y=y_train,
        batch_size=256,
        epochs=10,
        callbacks=[
            ModelCheckpoint('./models/%s.h5' % (start_time), monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
            ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, verbose=1, mode='auto')
        ],
        validation_data=(x_val, y_val),
        use_multiprocessing=True,
        workers=16      #Q. workers가 뭔가요? 
    )

Epoch 1/10
 199/4764 [>.............................] - ETA: 83:23:24 - loss: 0.0212 - acc: 0.0044

In [ ]:
start_time = datetime.now().strftime('%Y%m%d_%H%M%S')
os.makedirs('models', exist_ok=True)

model.fit(
    x=x_train,
    y=y_train,
    batch_size=256,
    epochs=10,
    callbacks=[
        ModelCheckpoint('./models/%s.h5' % (start_time), monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, verbose=1, mode='auto')
    ],
    validation_data=(x_val, y_val),
    use_multiprocessing=True,
    workers=16      #Q. workers가 뭔가요? 
)

Epoch 1/10
  32/4764 [..............................] - ETA: 105:41:02 - loss: 0.0697 - acc: 3.6621e-04

In [ ]:
i = 0

for y in range(h):
    for x in range(w):
        print('%2d' % x_val[i][y, x], end='')
    print()

In [ ]:
y_pred = model.predict(np.expand_dims(x_val[i], axis=0)).squeeze()
y_pred = y_pred.reshape((h, w))

y, x = np.unravel_index(np.argmax(y_pred), y_pred.shape)

print(x, y, y_pred[y, x])